In [212]:
class Value:
    def __init__(self, value, children=(), _op=""):
        self.data = value
        self.grad = 0
        self._prev = set(children)
        self._backward = lambda: None
        
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = backward
        return out

    def __pow__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data ** other.data, (self, other), '**')
        def backward():
            self.grad += other.data * (self.data ** (other.data - 1)) * out.grad
        out._backward = backward
        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')
        def backward():
            self.grad += (self.data > 0) * out.grad
        out._backward = backward
        return out

    def backward(self):
        total = []
        viewed = set()
        
        def top(v):
            if v not in viewed:
                viewed.add(v)
                for child in v._prev:
                    top(child)
                total.append(v)
        top(self)
        
        self.grad = 1
        for v in reversed(total):
            v._backward()
        
    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __truediv__(self, other):
        return self * (other ** -1)

    def __rtruediv__(self, other):
        return other * (self ** -1)

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"


In [213]:
# Primer složenijeg izračunavanja
a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)
d = a * b + c
e = Value(2.0)
f = d / e
g = f ** 2
h = g.relu()

# Izvršite backward propogaciju
h.backward()

print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("e:", e)
print("f:", f)
print("g:", g)
print("h:", h)


a: Value(data=2.0, grad=-6.0)
b: Value(data=-3.0, grad=4.0)
c: Value(data=10.0, grad=2.0)
d: Value(data=4.0, grad=2.0)
e: Value(data=2.0, grad=-4.0)
f: Value(data=2.0, grad=4.0)
g: Value(data=4.0, grad=1)
h: Value(data=4.0, grad=1)
